In [ ]:
import flwr as fl
import LR_Utils as utils
from flwr.common import NDArrays, Scalar
from sklearn.metrics import log_loss
from sklearn import metrics 
import pandas as pd
from sklearn.linear_model import LogisticRegression
from typing import Tuple, Dict, Optional

In [ ]:
def fit_round(server_round: int) -> Dict:
    """Send round number to client."""
    return {"server_round": server_round}

In [ ]:
def get_evaluate_fn(model: LogisticRegression):
    """Return an evaluation function for server-side evaluation."""

    _, (X_test, y_test) = utils.load_Data()

    def evaluate(
        server_round: int, parameters: NDArrays, config: Dict[str, Scalar]
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        utils.set_model_params(model, parameters)
        loss = log_loss(y_test, model.predict_proba(X_test))
        accuracy = model.score(X_test, y_test)
        return ({"Sever Loss":loss}, {"Server Accuracy": accuracy})
        
    return evaluate


In [ ]:
if __name__ == "__main__":
    model = LogisticRegression()
    utils.set_initial_params(model)
    strategy = fl.server.strategy.FedAvg(
        min_available_clients=2,
        evaluate_fn=get_evaluate_fn(model),
        on_fit_config_fn=fit_round,

    )

In [ ]:
fl.server.start_server(
    server_address="127.0.0.1:8080",
    strategy=strategy,
    config=fl.server.ServerConfig(num_rounds=300)
)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
trainDF = pd.read_csv(r"C:\Users\adamc\Work\HomeLab\ML Dissertation\Datasets\UNSW-NB15\UNSW_NB15_training-set.csv")
testDF = pd.read_csv(r"C:\Users\adamc\Work\HomeLab\ML Dissertation\Datasets\UNSW-NB15\UNSW_NB15_testing-set.csv")


In [ ]:
trainDF.shape

testDF.shape

In [ ]:
trainDF = trainDF.dropna()
trainDF = trainDF.drop_duplicates()

testDF = testDF.dropna()
testDF = testDF.drop_duplicates()

In [ ]:
#Removing Redundant Columns
trainDF = trainDF.drop(columns='id')
trainDF = trainDF.drop(columns='label')

testDF = testDF.drop(columns='id')
testDF = testDF.drop(columns='label')


In [ ]:
trainDF

In [ ]:
#Import one hot encoder from sklearn

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
enc.fit(trainDF[['proto','service','state','attack_cat']])  # Fit encoder on training data

train_encoded = enc.transform(trainDF[['proto','service','state','attack_cat']])
test_encoded = enc.transform(testDF[['proto','service','state','attack_cat']])

# Replace 'proto','service','state','attack_cat' column with encoded data
trainDF = pd.concat([trainDF.drop(['proto','service','state','attack_cat'], axis=1), pd.DataFrame(train_encoded, columns=enc.get_feature_names_out(['proto','service','state','attack_cat']))], axis=1)
testDF = pd.concat([testDF.drop(['proto','service','state','attack_cat'], axis=1), pd.DataFrame(test_encoded, columns=enc.get_feature_names_out(['proto','service','state','attack_cat']))], axis=1)
    


In [ ]:
trainDF.shape

In [ ]:
#Create classes variable listing all outcomes of dataset
class_name = ['attack_cat_Analysis',
 'attack_cat_Backdoor',
 'attack_cat_DoS',
 'attack_cat_Exploits',
 'attack_cat_Fuzzers',
 'attack_cat_Generic',
 'attack_cat_Normal',
 'attack_cat_Reconnaissance',
 'attack_cat_Shellcode',
 'attack_cat_Worms']
# Select everything other than classes
x_train = trainDF.drop(columns=class_name)
x_test = testDF.drop(columns=class_name)
# Select only classes
y_test = testDF[class_name]
y_train = trainDF[class_name]


In [ ]:
trainDF.shape

In [ ]:
y_train.shape

In [ ]:
import numpy as np
y_test = np.argmax(y_test,axis=1)
y_train = np.argmax(y_train,axis=1)

In [ ]:
y_test.shape

In [ ]:
unique, counts = np.unique(y_test, return_counts=True)
print("Unique values:", unique)
print("Counts:", counts)

In [ ]:
trainDF.shape

In [ ]:
testDF.shape

In [ ]:
x_test.shape

In [ ]:
x_train.shape

In [ ]:
y_test.shape

In [ ]:
y_train.shape

In [ ]:
target_columns = [col for col in trainDF.columns if col.startswith('attack_cat')]
target_columns

In [ ]:
#Imports minmaxscaler to normalise (Scale) data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaler.fit(trainDF)

trainDF = scaler.transform(trainDF)
testDF = scaler.transform(testDF)

In [ ]:
trainDF